In [1]:
import pandas as pd
from IPython.display import clear_output as clear

In [20]:
df = pd.read_excel("multi_label_move_predict.xlsx", "Sheet1", sep=";")
print(df.head())

   X_TL  X_TM  X_TR  X_ML  X_MM  X_MR  X_BL  X_BM  X_BR  Y_TL  Y_TM  Y_TR  \
0     1    -1     0     0     0     0     0     1     0     0     0     0   
1     1     0     1     0     0     0     0    -1     0     0     1     0   
2     0     0     0     0     0     1     0     0     0     0     0     1   
3     0     1     1     0     0     0     0    -1     0     1     0     0   
4     1     1    -1     0     0     0     0    -1     0     0     0     0   

   Y_ML  Y_MM  Y_MR  Y_BL  Y_BM  Y_BR  
0     0     0     0     1     0     1  
1     1     1     1     1     0     1  
2     1     1     0     0     0     1  
3     0     0     0     0     0     0  
4     1     0     1     1     0     1  


In [3]:
X = df.drop(['Y_TL' ,'Y_TM' ,'Y_TR' ,'Y_ML' ,'Y_MM' ,'Y_MR' ,'Y_BL' ,'Y_BM' ,'Y_BR'], axis=1)

In [4]:
y = df[['Y_TL', 'Y_TM', 'Y_TR', 'Y_ML',  'Y_MM', 'Y_MR', 'Y_BL',  'Y_BM', 'Y_BR']]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [6]:
X_train, x_test, Y_train, y_test = train_test_split(X, y, test_size=0.01)
model_dtc = DecisionTreeClassifier(random_state=13)
model_dtc.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=13,
            splitter='best')

In [7]:
y_pred = model_dtc.predict(x_test)
ans = accuracy_score(y_test, y_pred)
print("Accuracy = ", ans)

Accuracy =  0.9090909090909091


In [8]:
x_test.head(1)

,X_TL,X_TM,X_TR,X_ML,X_MM,X_MR,X_BL,X_BM,X_BR
240,-1,1,0,-1,0,1,0,-1,1


# Project implementation

In [9]:
#check rows
#return 10 if player 1 got trio in rows and -10 if player 2 get sames
def check_rows(a):
    for rows in a:
        cnt_p1 = 0
        cnt_p2 = 0
        for i in rows:
            if i == 1:
                cnt_p1+=1
            elif i == -1:
                cnt_p2+=1
        if cnt_p1 == 3: return 10
        elif cnt_p2 == 3: return -10
    return 0

In [10]:
#check cols
#return 20 if P1 won else return -20 if P2 won
def check_cols(a):
    for j in range(0, 3):
        cnt_p1 = 0
        cnt_p2 = 0
        for i in range(0, 3):
            if a[i][j] == 1: cnt_p1 += 1
            elif a[i][j] == -1: cnt_p2 += 1
            if cnt_p1 == 3: return 20
            elif cnt_p2 == 3:
                return -20
    return 0

In [11]:
#check diagonals if P1 win return 30 else if P2 won return -30
def check_diagonals(a):
    if a[0][0] == a[1][1] == a[2][2] == 1: return 30
    elif a[0][2] == a[1][1] == a[2][0] == 1: return 30
    elif a[0][0] == a[1][1] == a[2][2] == -1: return -30
    elif a[0][2] == a[1][1] == a[2][0] == -1: return -30
    else: return 0
    

In [12]:
def show_table(a):
    j = 0
    for i in a:
        print(i, '  ',end = '')
        j+=1
        if j == 3 : 
            print("")
            j = 0

In [13]:
#Converting 1-D arrya to 2-D array
#checking the trios for that array
def d1_d2(arr):
    trio_checker = 0
    d2_arr = []
    for i in range(0, 8, 3):
        temp = []
        for j in range(i, i+3):
            temp.append(arr[j])
        d2_arr.append(temp)
    #here we check for the trio 1st for rows if their is row trio than return it
    trio_checker = check_rows(d2_arr)
    if trio_checker != 0: return trio_checker
    else:
        #if not for rows than check for cols
        trio_checker = check_cols(d2_arr)
        if trio_checker != 0: return trio_checker
        else:
            #if not even for columns than check for diagonals
            trio_checker = check_diagonals(d2_arr)
            if trio_checker != 0: return trio_checker
            else: return 0

In [14]:
import time
df_for_append = pd.read_excel("multi_label_move_predict.xlsx", "Sheet1", sep=";")

In [15]:
def append_row(list1, list2):
    start = time.time()
    main_list = []
    global df_for_append
    cols_for_append = df_for_append.columns
    for i in range(0, 9):
        main_list.append(list1[i])
    for i in range(0, 9):
        main_list.append(list2[0][i])
    df2 = pd.DataFrame([main_list], columns=list(cols_for_append))
    df_for_append = df_for_append.append(df2)
    df_for_append.to_excel("multi_label_move_predict.xlsx", "Sheet1", index=False)
    elapsed_time = time.time() - start
    tot = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print("\nTime = ", tot)

In [16]:
#for all possibile index wher 1 comes in solution set
#stores the indexes in index_list check for the perfect_index to find the solution
#perfect index is one where the AI wins if no perfect index than returns 0th value
def keep_perfect_zero(predict_list, table):
    index_list = []
    for one in range(0, 9):
        if predict_list[0][one] == 1:
            index_list.append(one)
    perfect_index = int(0)
    for value in index_list:
        table[value] = -1
        perfect_index = d1_d2(table)
        if perfect_index < 0: return value
        else: table[value] = 0
    return index_list[0]

In [17]:
def check_box(table, index):
    if index > 8: return 1
    if table[index] != 0: return 1
    else: return 0

In [18]:
table_temp = [i for i in range(0, 9)]

In [19]:
x_list = [0 for i in range(0, 9)]
y_list = [0 for i in range(0, 9)]
table = [0 for i in range(0, 9)]
ox_table = ['-' for i in range(0, 9)]
ans = 0
print("Enter the value shown in box to keep 'X' ")
show_table(table_temp)
for i in range(0, 9):
    x_index = int(input("Enter box-value:- "))
    if x_index == 10: break
    while check_box(table, x_index):
        print("Entered box already has a value\nEnter other Box Number")
        x_index = int(input("Enter the box-value"))
    table[x_index] = 1
    x_list = table
    ox_table[x_index] = 'X'
    ans = d1_d2(table)
    if ans > 0:
        print("\nPlayer 1 won")
        break
    ans_list = model_dtc.predict([table])
    #if i >= 2: #append_row(x_list, ans_list)
    y_index = keep_perfect_zero(ans_list, table)
    table[y_index] = -1
    ox_table[y_index] = 'O'
    ans = d1_d2(table)
    clear()
    show_table(table_temp)
    print("\n")
    show_table(ox_table)
    if ans < 0:
        print("\nAI won")
        break
    zero_indicate = 0
    for box in table:
        if box == 0:
            zero_indicate = 1
            break
    if zero_indicate == 0: break
print("\nFinal Result")
if ans == 0: print("It's a draw")
show_table(ox_table)

0   1   2   
3   4   5   
6   7   8   


X   O   X   
O   O   O   
X   X   O   

AI won

Final Result
X   O   X   
O   O   O   
X   X   O   
